In [1]:
import os
import time
from math import floor, ceil

import numpy
from keras import layers, metrics
from keras import models, optimizers
from keras.applications import VGG16
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from utils import *
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

start = time.time()

train_dir = '/home/snowflake/Documents/nowe_trendy/train'
test_dir = '/home/snowflake/Documents/nowe_trendy/test'

path, dirs, files = os.walk(train_dir).__next__()

val_size = 0.15
nTrain = sum([len(files) for r, d, files in os.walk(train_dir)]) - len(dirs)
nVal = ceil(nTrain * val_size)
path, dirs, files = os.walk(test_dir).__next__()
nTest = sum([len(files) for r, d, files in os.walk(test_dir)]) - len(dirs)
nClasses = len(dirs)
batch_size = 20
dense_size = 256
epochs = 10

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print("**********Getting train features************")
all_features, all_labels = get_features_and_labels(nTrain,
                                                                  nClasses,
                                                                  batch_size,
                                                                  train_dir,
                                                                  conv_base,
                                                                  aug=False)

**********Getting train features************
Found 9694 images belonging to 12 classes.


/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [3]:
print("**********Getting augmented train features************")
aug_features, aug_labels = get_features_and_labels(nTrain,
                                                                  nClasses,
                                                                  batch_size,
                                                                  train_dir,
                                                                  conv_base,
                                                                  aug=True)

**********Getting augmented train features************
Found 9694 images belonging to 12 classes.


/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [4]:
all_features = np.concatenate((all_features, aug_features), 0)
all_labels = np.concatenate((all_labels, aug_labels), 0)

del aug_features
del aug_labels

In [5]:
val_size = 0.176
nVal = ceil(nTrain * val_size)

In [6]:
seed = 7
numpy.random.seed(seed)
train_features, validation_features, train_labels, validation_labels = train_test_split(all_features,
                                                                                        all_labels,
                                                                                        test_size=val_size,
                                                                                        random_state=seed)

In [43]:
epochs = 20
batch_size = 64
dense_size = 128

In [44]:
print("**********Configure model for training************")
model = models.Sequential()
model.add(layers.Dense(dense_size, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(nClasses, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=[metrics.categorical_accuracy])

**********Configure model for training************


In [45]:
print("**********Start of training************")
history = model.fit(train_features,
                    train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_features, validation_labels))

**********Start of training************
Train on 15975 samples, validate on 3413 samples
Epoch 1/20
15975/15975 [==============================] - 13s 838us/step - loss: 2.1818 - categorical_accuracy: 0.2597 - val_loss: 1.8958 - val_categorical_accuracy: 0.3841
Epoch 2/20
15975/15975 [==============================] - 13s 821us/step - loss: 1.8583 - categorical_accuracy: 0.3855 - val_loss: 1.7201 - val_categorical_accuracy: 0.4333
Epoch 3/20
15975/15975 [==============================] - 13s 824us/step - loss: 1.6802 - categorical_accuracy: 0.4356 - val_loss: 1.5961 - val_categorical_accuracy: 0.4659
Epoch 4/20
15975/15975 [==============================] - 13s 822us/step - loss: 1.5655 - categorical_accuracy: 0.4752 - val_loss: 1.5619 - val_categorical_accuracy: 0.4870
Epoch 5/20
15975/15975 [==============================] - 13s 823us/step - loss: 1.4761 - categorical_accuracy: 0.5048 - val_loss: 1.5408 - val_categorical_accuracy: 0.4887
Epoch 6/20
15975/15975 [======================

In [46]:
# nTest = 1821

In [47]:
# print("**********Getting test features************")
# test_features, test_labels = get_features_and_labels(nTest,
#                                                                      nClasses,
#                                                                      batch_size,
#                                                                      test_dir,
#                                                                      conv_base,
#                                                                     aug = False)

In [48]:
print("**********End of training************\n\n")

ground_truth = [np.where(r == 1)[0][0] for r in test_labels]

predictions = model.predict_classes(test_features)
prob = model.predict(test_features)

# errors = nVal - accuracy_score(ground_truth, predictions, normalize=False)
errors = np.where(predictions != ground_truth)[0]
accuracy = accuracy_score(ground_truth, predictions)
precision, recall, f_score, support = score(ground_truth, predictions, average='weighted')

print("No of errors = {}/{}".format(len(errors), nTest))
print("Accuracy: {}".format(accuracy))
print("Weighted average Precision: {}".format(precision))
print("Weighted average Recall: {}".format(recall))
print("Weighted average F-score: {}".format(f_score))

end = time.time()
print("Time: " + str(end - start))


**********End of training************


No of errors = 758/1821
Accuracy: 0.5837451949478308
Weighted average Precision: 0.6073077208784619
Weighted average Recall: 0.5837451949478308
Weighted average F-score: 0.5844704216069765
Time: 20031.195976257324


In [49]:
# f = open("/media/snowflake/Data/features.dat", "wb+")
# np.save(f, all_features)

In [50]:
# f = open("/media/snowflake/Data/labels.dat", "wb+")
# np.save(f, all_labels)